<a href="https://colab.research.google.com/github/msabid/kfc-kingston-sentiment-analysis/blob/main/kfc_kingston_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Calling

In [ ]:
import urllib.parse
import requests
import pprint
import pandas as pd
from google.colab import userdata
from google.cloud import language_v1
from google.colab import drive

# Data Extraction

In [ ]:
# Get Google Maps API key
googleMapsKey = userdata.get('google-maps-key')

In [ ]:
# Function for finding the place
def geoCodePlace(query, location, radius):
    """Returns the places for the given query and location."""
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "keyword": query,
        "location": location,
        "radius": radius,
        "key": googleMapsKey
    }
    response = requests.get(url, params=urllib.parse.urlencode(params))
    return response.json()

# Function for getting place details
def getPlaceDetails(place_id):
    """Returns the place details for the given place id."""
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "fields": "place_id,name,rating,user_ratings_total,reviews,vicinity",
        "key": googleMapsKey,
    }
    response = requests.get(url, params=urllib.parse.urlencode(params))
    return response.json()


# Function to analyze sentiment
def analyze_sentiment(text):
    document = language_v1.types.Document(
        content=text, type_=language_v1.types.Document.Type.PLAIN_TEXT
    )
    sentiment = client.analyze_sentiment(
        request={"document": document}
    ).document_sentiment
    return sentiment.score, sentiment.magnitude

In [ ]:
# Define the query and location
query = "KFC"
location = "44.2312,-76.4860"  # Latitude and longitude of Kingston, Ontario
radius = 10000  # Search radius in meters

In [ ]:
# Get the geoCodedPlace
geoCodedPlace = geoCodePlace(query, location, radius)
print("\nThese are the KFC locations in Kingston, Ontario:")
pprint.pprint(geoCodedPlace)


These are the KFC locations in Kingston, Ontario:
{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 44.2505023, 'lng': -76.5306963},
                           'viewport': {'northeast': {'lat': 44.25183257989273,
                                                      'lng': -76.52932372010729},
                                        'southwest': {'lat': 44.24913292010729,
                                                      'lng': -76.53202337989272}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
              'icon_background_color': '#FF9E67',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
              'name': 'KFC',
              'opening_hours': {'open_now': False},
              'photos': [{'height': 1920,
                          'html_attributions': ['<a '
                             

In [ ]:
# Extract place_ids
place_ids = [place['place_id'] for place in geoCodedPlace['results']]
print("\nThe place ids are:")
pprint.pprint(place_ids)


The place ids are:
['ChIJ15h7AXes0kwRB6vBYdACEY0',
 'ChIJT1mRDEus0kwRj6busiW3kUo',
 'ChIJ6Z8PE4es0kwRSmh1CnDGRX0',
 'ChIJW4n0HeSr0kwRO0pPiPa4xeM']


In [ ]:
# Initialize a list to store review data
review_data = []

# Get place details for each place_id
for place_id in place_ids:
    placeDetails = getPlaceDetails(place_id)
    print("\nThese are the place details:")
    pprint.pprint(placeDetails)

    if 'result' in placeDetails and 'reviews' in placeDetails['result']:
        reviews = placeDetails['result']['reviews']
        for i in range(min(3, len(reviews))):  # Ensure we only attempt to access up to 3 reviews
            review = reviews[i]
            publish_time = review.get('time', 0)
            if publish_time:
                publish_time = pd.to_datetime(publish_time, unit='s').isoformat() + 'Z'
            data = {
                'place_id': place_id,
                'publishTime': publish_time,
                'DisplayName': review.get('author_name', ''),
                'OriginalText': review.get('text', ''),
                'Rating': review.get('rating', '')
            }
            review_data.append(data)
            print(f"Appended review data: {data}")
    else:
        print(f"No reviews found for place_id {place_id}")



These are the place details:
{'html_attributions': [],
 'result': {'name': 'KFC',
            'place_id': 'ChIJ15h7AXes0kwRB6vBYdACEY0',
            'rating': 3,
            'reviews': [{'author_name': 'Kate Gollogly',
                         'author_url': 'https://www.google.com/maps/contrib/114555211384266547567/reviews',
                         'language': 'en',
                         'original_language': 'en',
                         'profile_photo_url': 'https://lh3.googleusercontent.com/a/ACg8ocJ1Q-Z_IpMD4VROCfsvGkM4BCxcu7_3p8zMTwxqEIvXdWwGTA=s128-c0x00000000-cc-rp-mo-ba6',
                         'rating': 4,
                         'relative_time_description': 'a month ago',
                         'text': 'Visited on a Tuesday around 5pm, restaurant '
                                 'was not overly busy.  I ordered two popcorn '
                                 'chicken meals. The popcorn chicken was '
                                 'freshly made, as I had to wait 

In [ ]:
# Create a DataFrame from the review data
df = pd.DataFrame(review_data)
df

,place_id,publishTime,DisplayName,OriginalText,Rating
0,ChIJ15h7AXes0kwRB6vBYdACEY0,2024-04-20T08:48:06Z,Kate Gollogly,"Visited on a Tuesday around 5pm, restaurant wa...",4
1,ChIJ15h7AXes0kwRB6vBYdACEY0,2023-10-29T02:08:59Z,Karim Merchant,Chicken was stale didn’t seem fresh. Color was...,2
2,ChIJ15h7AXes0kwRB6vBYdACEY0,2024-03-15T22:46:03Z,Moiz Master,The worst burger I have ever had in my life. I...,1
3,ChIJT1mRDEus0kwRj6busiW3kUo,2024-01-30T18:04:15Z,YaISaidIt,"KFC at the mall for fried chicken, fries, grav...",2
4,ChIJT1mRDEus0kwRj6busiW3kUo,2023-11-17T21:04:18Z,Aivy Faye de Guzman-Bonabon,Terrible experience about survey promotions.\n...,1
5,ChIJT1mRDEus0kwRj6busiW3kUo,2023-10-23T19:39:10Z,Vodka Toxic,I found the price very high for just 10 chicke...,1
6,ChIJ6Z8PE4es0kwRSmh1CnDGRX0,2023-12-07T03:34:36Z,Kat J,This KFC is combined with a Taco Bell but you ...,3
7,ChIJ6Z8PE4es0kwRSmh1CnDGRX0,2022-03-09T22:32:56Z,First Last,Ordered a bucket of chicken with sides and pop...,1
8,ChIJ6Z8PE4es0kwRSmh1CnDGRX0,2023-12-31T01:06:58Z,Psycho Boo,I miss when KFC used to be good and it didn't ...,1
9,ChIJW4n0HeSr0kwRO0pPiPa4xeM,2024-05-20T20:54:31Z,Bibin Chacko,"The chicken was well made , good to try. But I...",5


In [ ]:
# Convert 'publishTime' to datetime format
df['publishTime'] = pd.to_datetime(df['publishTime'])
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   place_id      12 non-null     object             
 1   publishTime   12 non-null     datetime64[ns, UTC]
 2   DisplayName   12 non-null     object             
 3   OriginalText  12 non-null     object             
 4   Rating        12 non-null     int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(3)
memory usage: 608.0+ bytes


In [ ]:
# Initialize a list to store place details
place_details_data = []

# Get place details for each place_id and store required fields
for place_id in place_ids:
    placeDetails = getPlaceDetails(place_id)
    if 'result' in placeDetails:
        result = placeDetails['result']
        data = {
            'place_id': result.get('place_id', ''),
            'vicinity': result.get('vicinity', ''),
            'user_ratings_total': result.get('user_ratings_total', 0)
        }
        place_details_data.append(data)
        print(f"Appended place details: {data}")
    else:
        print(f"No details found for place_id {place_id}")

Appended place details: {'place_id': 'ChIJ15h7AXes0kwRB6vBYdACEY0', 'vicinity': '1407 Princess Street, Kingston', 'user_ratings_total': 379}
Appended place details: {'place_id': 'ChIJT1mRDEus0kwRj6busiW3kUo', 'vicinity': '945 Gardiners Road, Kingston', 'user_ratings_total': 91}
Appended place details: {'place_id': 'ChIJ6Z8PE4es0kwRSmh1CnDGRX0', 'vicinity': '1668 Bath Road, Kingston', 'user_ratings_total': 148}
Appended place details: {'place_id': 'ChIJW4n0HeSr0kwRO0pPiPa4xeM', 'vicinity': '29 Warne Crescent, Kingston', 'user_ratings_total': 558}


In [ ]:
df2 = pd.DataFrame(place_details_data)
df2

,place_id,vicinity,user_ratings_total
0,ChIJ15h7AXes0kwRB6vBYdACEY0,"1407 Princess Street, Kingston",379
1,ChIJT1mRDEus0kwRj6busiW3kUo,"945 Gardiners Road, Kingston",91
2,ChIJ6Z8PE4es0kwRSmh1CnDGRX0,"1668 Bath Road, Kingston",148
3,ChIJW4n0HeSr0kwRO0pPiPa4xeM,"29 Warne Crescent, Kingston",558


# Sentiment Analysis

In [ ]:
# Initialize Google Cloud SDK
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  Y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthc

In [ ]:
# Enable Google Cloud Natural Language API
!gcloud services enable language.googleapis.com

In [ ]:
# Authenticate to use Google Cloud services
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=orywK9QnAkJZa1rDg26V1DxuRDLPNZ&prompt=consent&token_usage=remote&access_type=offline&code_challenge=VvL1IOYIdFshylqrIuZbvfXQRD_QL4PerIdGHV1wQMQ&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AdLIrYch0y3q5v85LU_51Xx9Y1zva0uwVRChnHfCK_Hg9uHs9xp9nCmasFcmShWq0Azhtg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Q

In [ ]:
# Instantiates a client
client = language_v1.LanguageServiceClient()

In [ ]:
# Apply sentiment analysis to each review
df['Sentiment_Score'] = df['OriginalText'].apply(lambda text: analyze_sentiment(text)[0])
df['Sentiment_Magnitude'] = df['OriginalText'].apply(lambda text: analyze_sentiment(text)[1])


In [ ]:
df.head()

,place_id,publishTime,DisplayName,OriginalText,Rating,Sentiment_Score,Sentiment_Magnitude
0,ChIJ15h7AXes0kwRB6vBYdACEY0,2024-04-20 08:48:06+00:00,Kate Gollogly,"Visited on a Tuesday around 5pm, restaurant wa...",4,0.3,1.9
1,ChIJ15h7AXes0kwRB6vBYdACEY0,2023-10-29 02:08:59+00:00,Karim Merchant,Chicken was stale didn’t seem fresh. Color was...,2,-0.3,2.3
2,ChIJ15h7AXes0kwRB6vBYdACEY0,2024-03-15 22:46:03+00:00,Moiz Master,The worst burger I have ever had in my life. I...,1,-0.3,4.6
3,ChIJT1mRDEus0kwRj6busiW3kUo,2024-01-30 18:04:15+00:00,YaISaidIt,"KFC at the mall for fried chicken, fries, grav...",2,-0.3,4.7
4,ChIJT1mRDEus0kwRj6busiW3kUo,2023-11-17 21:04:18+00:00,Aivy Faye de Guzman-Bonabon,Terrible experience about survey promotions.\n...,1,-0.7,10.7


# Exporting Data as CSV

In [ ]:
# Mount Google Drive to save the data
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Export df and df2 as separate CSV files
df.to_csv('/content/drive/My Drive/ML&AI/store_data_reviews.csv', index=False)
df2.to_csv('/content/drive/My Drive/ML&AI/store_data_place_details.csv', index=False)